In [1]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Dense, Activation, Input, Embedding, Reshape
from keras.layers import LSTM, GRU, Conv1D
from keras.layers.wrappers import TimeDistributed
from keras.activations import sigmoid
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.callbacks import TensorBoard
import numpy as np
import random
import sys
import os

Using TensorFlow backend.


In [2]:
import re

def split_raw_into_sentences(text):
    symbs = re.compile(r"[^А-Яа-я:!\?,\.\"— - \n]")
    text = re.sub(symbs, "", text)

    text = re.sub(r'\*|\x01|\xa0|--|\t|"|(\[.*\])', "", text)
    text = re.sub(r'[A-Za-z]+[!.,;:\-?]*', "", text)
    text = re.sub(r"\n[ ]+", "\n", text)
    text = re.sub(r"[ ]+", " ", text)
    text = re.sub(r'(?<=[А-Яа-я,;:\—" ])\n(?=[а-яPS])', r" ", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"(?<=[А-Яа-я])[ ]*\n", r".\n", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"\n(?! \— [а-я])", "", text)
    text = re.sub(r"([!?.,:;]+)(?=[А-Яа-я])", r"\1 ", text)
    text = re.sub(
        r"(?<!\W[А-Я]\.)(?<!\WP\.)(?<!\WP\. S\.)(?<!\W(т|Т)\.)(?<!\W(т|Т)\. (к|К|д|Д)\.)(?<=\.|\?|\!)(?! [а-я])(?! \— [а-я])\s",
        "\n", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r";", ",", text)
    text = re.sub(r"\—(?=.)", "— ", text)
    text = re.sub(r"([!?.,:;]+)(?=[А-Яа-я])", r"\1 ", text)
    text = re.sub(r"(?<=[!?.,:;])\—", " —", text)
    text = re.sub(r"\.{4,}", "", text)
    text = re.sub(r"\s\.\s", "\n", text)
    text = re.sub(r"(?<=[!.,?:;])P.", "\nP.", text)
    text = re.sub(r"(?<=[!.,?:;])\— (?![а-я])", "\n—", text)
    text = re.sub(
        r"(?<!\W[А-Я]\.)(?<!\WP\.)(?<!\WP\. S\.)(?<!\W(т|Т)\.)(?<!\W(т|Т)\. (к|К|д|Д)\.)(?<=\.|\?|\!)(?! [а-я])(?! \— [а-я])\s",
        "\n", text)
    text = re.sub(r"(?<=[!?.,:;])\n\— (?=[а-я])", " — ", text)
    text = re.sub(r"\([\d]*\)", "", text)
    text = re.sub(r"(\.\n){2,}", "\n", text)
    text = re.sub(r"[IXV]+", r"", text)
    text = re.sub(r"\s\.\s", "\n", text)
    text = re.sub(r"\s([!?.,:;])", "\1", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"\n[ ]+", "\n", text)
    text = re.sub(r"[ ]+", " ", text)
    text = re.sub(r"(?<=[!?.,:;])\n\— (?=[а-я])", " — ", text)
    return text

In [3]:
from libs.utils import text_preprocess

In [4]:
with open('data/dostoewskij.txt', encoding='utf-8') as f:
    dostoewskij_text = f.read().lower()
with open('data/non_dostoewskij_texts.txt', encoding='utf-8') as f:
    non_dostoewskij_text = f.read().lower()

sentenced_dostoewskij = split_raw_into_sentences(dostoewskij_text)
sentenced_non_dostoewskij = split_raw_into_sentences(non_dostoewskij_text)

sentenced_dostoewskij = text_preprocess(sentenced_dostoewskij)
sentenced_non_dostoewskij = text_preprocess(sentenced_non_dostoewskij)

dostoewskij_sentences = sentenced_dostoewskij.split('\n')
non_dostoewskij_sentences = sentenced_non_dostoewskij.split('\n')

#dostoewskij_text = clear_text_from_rare_chars(dostoewskij_text, delete_enters=True)
#non_dostoewskij_text = clear_text_from_rare_chars(non_dostoewskij_text, delete_enters=True)

print('dostoewskij_length:\t', len(dostoewskij_text))
print('non_dostoewskij_length:\t', len(non_dostoewskij_text))

dostoewskij_length:	 4967030
non_dostoewskij_length:	 175060228


In [5]:
s = set(sentenced_dostoewskij)
print(len(s))
s2 = set(sentenced_non_dostoewskij)
print(len(s2))

39
39


In [6]:
from libs.utils import load_transformer

transformer = load_transformer('models/shm_c1')

chars = transformer.tokens
char_cats = len(chars)
print('total chars:', char_cats)

total chars: 42


In [7]:
print_len = 2000
pos = np.random.randint(len(dostoewskij_text))
print(dostoewskij_text[pos:pos+print_len])
print('-' * 100)
pos = np.random.randint(len(non_dostoewskij_text))
print(non_dostoewskij_text[pos:pos+print_len])

едоразвитой и вовсе не 

учащейся молодежи". не знаю, что именно хотел доказать этим вывертом собственно 

автор статейки "русского мира": хотел ли он польстить учащейся молодежи? или, 

напротив, хитрым маневром и, так сказать, в виде ласкательства думал ее же 

поднадуть немного, но только с самыми почтенными целями, - то есть для ее же 

пользы, - и для достижения цели употребил столь известный прием гувернанток и 

нянюшек с маленькими ребятками: вот, дескать, милые дети, видите, какие те 

нехорошие буяны, кричат и дерутся, и их непременно высекут за то, что они такие 

"недоразвитки"; вы же вот такие милые хваленые паиньки, за столом сидите прямо, 

ножками под столом не болтаете, и вам за это непременно гостинцу дадут. или, 

наконец, просто-запросто автору захотелось "защитить" нашу учащуюся молодежь 

перед правительством и употребить для сего прием, который сам он, может быть, 

считает необыкновенно хитрым и тонким? 

прямо скажу: хотя я поставил все эти вопросы, но личные п

In [8]:
n_batches = len(dostoewskij_text) // 3000
batch_size = 16
#n_batches -= n_batches % batch_size
max_len = 200

In [9]:
from libs.utils import pad

# transform text into sequence of indices
pad_idx = char_cats
dostoewskij_indexes     = np.array([pad(transformer.transform(sent), max_len, pad_idx) for sent in dostoewskij_sentences])
non_dostoewskij_indexes = np.array([pad(transformer.transform(sent), max_len, pad_idx) for sent in non_dostoewskij_sentences])

In [12]:
def split_data_into_correct_batches_sentences(text1_indexes, text2_indexes, make_equal_folding = True):
    prime_number = 2147483647
    
    X = np.zeros((n_batches, max_len), dtype=np.int64)
    Y = np.zeros((n_batches,), dtype=np.int64)
    
    choose_from_first = True
    index1 = 0
    index2 = 0
    for i in range(n_batches):
        if make_equal_folding:
            if choose_from_first:
                index1 = (index1 + prime_number) % (len(text1_indexes))
                X[i, :] = text1_indexes[index1]
                Y[i] = 0
            else:
                index2 = (index2 + prime_number) % (len(text2_indexes))
                X[i, :] = text2_indexes[index2]
                Y[i] = 1
                
            choose_from_first = not choose_from_first
        else:
            index1 = (index1 + prime_number) % (len(text1_indexes) + len(text2_indexes))
            if index1 < len(text1_indexes) - max_len + 1:
                X[i, :] = text1_indexes[index1]
                Y[i] = 0
            else:
                index2 = index1 - (len(text1_indexes))
                X[i, :] = text2_indexes[index2]
                Y[i] = 1
    return X, Y

X, y = split_data_into_correct_batches_sentences(dostoewskij_indexes, non_dostoewskij_indexes, make_equal_folding=True)

In [14]:
def split_data_into_correct_batches_raw(text1_indexes, text2_indexes, make_equal_folding = True):
    prime_number = 2147483647
    
    X = np.zeros((n_batches, max_len), dtype=np.int64)
    Y = np.zeros((n_batches,), dtype=np.int64)
    
    choose_from_first = True
    index1 = 0
    index2 = 0
    for i in range(n_batches):
        if make_equal_folding:
            if choose_from_first:
                index1 = (index1 + prime_number) % (len(text1_indexes) - max_len + 1)
                X[i, :] = text1_indexes[index1:index1+max_len]
                Y[i] = 0
            else:
                index2 = (index2 + prime_number) % (len(text2_indexes) - max_len + 1)
                X[i, :] = text2_indexes[index2:index2+max_len]
                Y[i] = 1
                
            choose_from_first = not choose_from_first
        else:
            index1 = (index1 + prime_number) % (len(text1_indexes) + len(text2_indexes) - 2*max_len + 2)
            if index1 < len(text1_indexes) - max_len + 1:
                X[i, :] = text1_indexes[index1:index1 + max_len]
                Y[i] = 0
            else:
                index2 = index1 - (len(text1_indexes) - max_len + 1)
                X[i, :] = text2_indexes[index2:index2 + max_len]
                Y[i] = 1
    return X, Y

X, y = split_data_into_correct_batches_raw(dostoewskij_indexes, non_dostoewskij_indexes, make_equal_folding=True)

ValueError: could not broadcast input array from shape (200,200) into shape (200)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

a = plt.hist(y)

In [ ]:
from keras.losses import sparse_categorical_crossentropy

In [ ]:
def create_char_rnn():
    inp = Input(shape=(max_len,), dtype="int32")
    v = Embedding(char_cats, int(char_cats / 1.5))(inp)
    h1 = GRU(256, stateful=False, return_sequences=True, unroll=True, implementation=0)(v)
    h2 = GRU(256, stateful=False, return_sequences=False, unroll=True, implementation=0)(h1)
    y = Dense(2, activation='softmax')(h2)
    model = Model(inp, y, name="char_rnn")
    model.compile(optimizer=RMSprop(), loss=sparse_categorical_crossentropy, metrics=['accuracy'])
    return model

In [ ]:
rnn = create_char_rnn()

In [ ]:
print(rnn.summary())

In [ ]:
history = rnn.fit(X, y, batch_size=batch_size, shuffle=True, epochs=1)#, callbacks=[tb])

In [ ]:
n_epochs = 8
histories = []
for epoch in range(n_epochs):
    X, y = split_data_into_correct_batches(dostoewskij_indexes, non_dostoewskij_indexes, make_equal_folding=True)
    histories.append(rnn.fit(X, y, batch_size=batch_size, shuffle=True, epochs=1))

In [ ]:
rnn.save('models/discriminator_style_rnn_model.h5')